In [1]:
import pandas as pd
import numpy as np
import json
import os
from tqdm import tqdm
import requests
import time
import pickle

In [2]:
metadataDictionary = {}

def metadata(cik):
    '''
    Given a cik, it retrieves a dataframe with the metadata of that security for each 10Q and 10K form.
    Parameters:
        CIK: sec unique and non-reusable SEC identifier
    Global variables:
        metadataDictionary: if the metadata for a given CIK has been retrieved before, it will just return it instead of getting
        it again
    Returns:
        DataFrame with metadata
    '''
    cik = str(cik)
    if len(cik) != 10:
        cik = cik.zfill(10)
    if cik not in metadataDictionary:
        path = 'https://data.sec.gov/submissions/CIK{}.json'.format(cik)
        user_agent = 'jdjp_97@hotmail.com'
        headers = {'User-Agent':user_agent} 

        metadata = requests.get(path, headers=headers).json()
        filings = metadata['filings']
        recent = pd.DataFrame( filings['recent'] )
        recent = recent[recent['form'].isin(['10-Q', '10-K'])]
        files = filings['files']

        dataFrames = [recent]
        #Loop thourgh files
        for idx, i in enumerate(files):
            fileName = i['name']
            path2 = 'https://data.sec.gov/submissions/{}'.format(fileName)
            recent2 = pd.DataFrame(requests.get(path2, headers = headers).json())
            recent2 = recent2[recent2['form'].isin(['10-Q', '10-K'])]
            if (idx % 9 == 0): #There is a time limit in requests. 10 per second
                time.sleep(1)

            dataFrames.append(recent2)

        finalDf = pd.concat(dataFrames).reset_index(drop = True)
        
        finalDf['url'] = 'https://www.sec.gov/Archives/edgar/data/'
        finalDf['url'] += cik + '/' + finalDf['accessionNumber'].str.replace('-', '') + '/' + finalDf['primaryDocument']
        
        finalDf['url_txt'] = 'https://www.sec.gov/Archives/edgar/data/'
        finalDf['url_txt'] += str(int(cik)) + '/' + finalDf['accessionNumber'].str.replace('-', '') + '/' + finalDf['accessionNumber'] + '.txt'
        
        finalDf['filingDate'] = pd.to_datetime(finalDf['filingDate'])
        finalDf['reportDate'] = pd.to_datetime(finalDf['reportDate'])
        
        metadataDictionary[cik] = finalDf
        
    else:
        finalDf = metadataDictionary[cik]
    
    return finalDf
    

# Computing CIKS
For the URL construction, we need CIKs.


- We take the file provided as input **sp500_w_addl_id.csv**

- The stock level data is extracted in **returns.csv** from https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/stock-security-files/daily-stock-file/. Using this data, we can get a mapping from permno -> cusip per given date. This is needed as permno is unique for a share class but cusip number changes over time. 

- We join this **sp500_w_addl_id.csv** data with returns data to add *cusip* column to our dataset.

- We downloaded data mapping gvkey to cik for different dates from https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/crspcompustat-merged/compustat-crsp-link/ and  do a merge asof over the nearest dates with the above data to add CIK.

In [ ]:
df = pd.read_csv('sp500_w_addl_id.csv', parse_dates = [1,11, 12]).iloc[:, 1:]
df['ncusip'] = df['ncusip'].astype(str)
df['date'] = df.date + pd.offsets.MonthEnd(0) 
df['permno'] = df['permno'].astype(np.int64)
print('Data Types')
print(df.dtypes)
print()
print('Shape')
print(df.shape)
df.head(10)

In [4]:
dtype_dic= {'CUSIP':str, 'NCUSIP':str}
returns = pd.read_csv('returns.csv', parse_dates = [1], dtype = dtype_dic)
returns.rename({'PERMNO': 'permno'}, inplace = True, axis = 1)
returns = returns.sort_values(by = 'date')
print(returns.dtypes)
print(returns.head())

permno               int64
date        datetime64[ns]
EXCHCD               int64
NCUSIP              object
TICKER              object
COMNAM              object
PRIMEXCH            object
TRDSTAT             object
PERMCO               int64
CUSIP               object
RCRDDT              object
NWPERM             float64
PRC                float64
VOL                float64
CFACPR             float64
OPENPRC            float64
dtype: object
         permno       date  EXCHCD    NCUSIP TICKER  \
0         10104 2011-01-03       3  68389X10   ORCL   
73198     12076 2011-01-03       1  G8127610    SIG   
1261958   82618 2011-01-03       3  80004C10   SNDK   
1263307   82642 2011-01-03       1  51843910     EL   
1266076   82643 2011-01-03       1  52977110    LXK   

                                COMNAM PRIMEXCH TRDSTAT  PERMCO     CUSIP  \
0                          ORACLE CORP        Q       A    8045  68389X10   
73198              SIGNET JEWELERS LTD        N       A    9857  G812

In [5]:
dtype_dic= {'iid':str, 'cusip':str}
mapping = pd.read_csv('finalMapping.csv', parse_dates = [2], dtype = dtype_dic)
mapping.rename({'datadate':'date'}, axis = 1, inplace = True)
mapping = mapping.sort_values('date')
mapping['cusip'] = mapping['cusip'].str[:8]
print(mapping.dtypes)
mapping.head()

gvkey               int64
iid                object
date       datetime64[ns]
tic                object
cusip              object
conm               object
divd              float64
exchg               int64
secstat            object
cik               float64
dtype: object


,gvkey,iid,date,tic,cusip,conm,divd,exchg,secstat,cik
0,1045,01,2011-01-03,AAMRQ,00176510,AMERICAN AIRLINES GROUP INC,NaN,19,I,6201.0
1549807,29099,05,2011-01-03,REG.PD,75884950,REGENCY CENTERS CORP,NaN,11,I,910606.0
1550121,29099,06,2011-01-03,REG.PE,75884960,REGENCY CENTERS CORP,NaN,11,I,910606.0
121037,2285,01,2011-01-03,BA,09702310,BOEING CO,NaN,11,A,12927.0
1553077,29127,01,2011-01-03,INCY,45337C10,INCYTE CORP,NaN,14,A,879169.0


In [10]:
merged = pd.merge_asof(df, returns, on = 'date', by ='permno', direction = 'backward')
merged.rename({'CUSIP':'cusip'}, inplace = True, axis = 1)
print(merged.isna().sum())
merged.head()

date            0
permno          0
comnam          0
ncusip          0
shrcd           0
exchcd          0
hsiccd          0
ticker          0
gvkey           0
iid             0
start           0
ending          0
ret            34
EXCHCD          0
NCUSIP          0
TICKER          0
COMNAM          0
PRIMEXCH        0
TRDSTAT         0
PERMCO          0
cusip           0
RCRDDT      65047
NWPERM      66100
PRC             0
VOL             0
CFACPR          0
OPENPRC         0
dtype: int64


,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,...,PRIMEXCH,TRDSTAT,PERMCO,cusip,RCRDDT,NWPERM,PRC,VOL,CFACPR,OPENPRC
0,2011-01-31,60986,NEWELL RUBBERMAID INC,65122910,11.0,1.0,3089.0,NWL,7875,01,...,N,A,3178,65122910,NaN,NaN,19.25,3631600.0,1.000000,19.43
1,2011-01-31,85914,BEST BUY COMPANY INC,08651610,11.0,1.0,5731.0,BBY,2184,01,...,N,A,7506,08651610,NaN,NaN,34.00,6034700.0,1.000000,34.01
2,2011-01-31,80711,APARTMENT INVESTMENT & MGMT CO,03748R10,18.0,1.0,6798.0,AIV,30490,01,...,N,A,30438,03748R74,NaN,NaN,25.56,1627300.0,8.766256,25.08
3,2011-01-31,59176,AMERICAN EXPRESS CO,02581610,11.0,1.0,6141.0,AXP,1447,01,...,N,A,90,02581610,NaN,NaN,43.38,9931500.0,1.000000,44.13
4,2011-01-31,75100,TIFFANY & CO NEW,88654710,11.0,1.0,5944.0,TIF,13646,01,...,N,A,21753,88654710,NaN,NaN,58.13,966300.0,1.000000,57.80


We fill in the missing CIK values using a manual dictionary.

In [ ]:
final = pd.merge_asof(merged, mapping, on = 'date', by ='gvkey', direction = 'nearest')
dict_ = {'APARTMENT INVESTMENT & MGMT CO': 922864,
 'ALCOA INC': 1675149,
 'PEABODY ENERGY CORP': 1064728,
 'CHESAPEAKE ENERGY CORP': 895126,
 'JACOBS ENGINEERING GROUP INC': 52988,
 'JANUS CAP GROUP INC': 1065865,
 'METROPCS COMMUNICATIONS INC': 1121026,
 'WISCONSIN ENERGY CORP': 783325,
 'RADIOSHACK CORP': 81727,
 'OWENS ILL INC': 861326,
 'NEWS CORP': 1564708,
 'WINDSTREAM CORP': 1282266,
 'WYNDHAM WORLDWIDE CORP': 1010471,
 'J D S UNIPHASE CORP': 1257179,
 'CONSOL ENERGY INC': 1710366,
 'YAHOO INC': 1426140,
 'DEAN FOODS CO': 931336,
 'DONNELLEY R R & SONS CO': 29669,
 'FRONTIER COMMUNICATIONS CORP': 20520,
 'SARA LEE CORP': 1080839,
 'I T T CORP': 216228,
 'NABORS INDUSTRIES LTD': 1163739,
 'SUPERVALU INC': 95521,
 'TWENTY FIRST CENTURY FOX INC': 1308161,
 'WINDSTREAM HOLDINGS INC': 1282266,
 'S L GREEN REALTY CORP': 1040971,
 'W E C ENERGY GROUP INC': 783325,
 'FEDERAL REALTY INVESTMENT TRUST': 34903,
 'G G P INC': 895648,
 'DISCOVERY INC': 1437107,
 'HOLLYFRONTIER CORP': 48039, 'CLIFFS NATURAL RESOURCES INC': np.nan, 'DEVRY INC DEL':np.nan, 
         'M E M C ELECTRONIC MATERIALS INC':np.nan, 'COCA COLA ENTERPRISES INC NEW': np.nan, 
         'DISCOVERY COMMUNICATIONS INC':np.nan, 'DR PEPPER SNAPPLE GROUP INC': np.nan, 'DEAN FOODS CO NEW': np.nan,
         'PENNEY J C CO INC':np.nan, 'E N S C O PLC NEW':np.nan, 'GENERAL GROWTH PPTYS INC NEW': np.nan
    
        }

temp = final[final.cik.isna()]
temp['cik'] = temp.apply(lambda row: dict_[row['comnam']] if np.isnan(row['cik']) else row['cik'], axis=1)
temp[['comnam', 'cik']]
final[final.cik.isna()] = temp
final = final.dropna(subset=['cik'])
del temp

print(final.shape)
final.isna().sum()

In [ ]:
uniqueCIKs = final.dropna(subset = ['cik']).cik.unique()
urlList = []
for cik in uniqueCIKs:
    try:
        cik_search = str(int(cik)).zfill(10)

        meta = metadata(cik_search)
        meta['reportDate2'] = meta['reportDate'].dt.strftime('%Y-%m')

        tempDf = final[final['cik'] == cik]
        tempDf['date2'] = tempDf['date'].dt.strftime('%Y-%m')

        mergedTempDf = pd.merge(tempDf, meta, left_on = 'date2', right_on = 'reportDate2')
        urlList.append(mergedTempDf)
    except:
        print('{} not found'.format(cik))
    
finalUrls = pd.concat(urlList)
finalUrls.head()

In [ ]:
finalUrls.to_csv('finalUrls.csv', index = False)